# Add your own providers to EODAG

Even though EODAG natively supports the connection to multiple providers, it is built as an extensible framework. It means that you can interface it with your own providers. This is what we are going to demonstrate in this section.

Those most straightforward way to achieve this is to reuse the existing plugins and configure them for your new provider.

1. [Add a provider in EODAG](../../add_provider.rst)

2. You can also [dynamically modify the providers](../tutos/tuto_stac_client.ipynb) of an existing EODAG instance. 

EODAG is shipped with plugins to support a wide range of known and standard API. But, you may have a provider with a custom authentication method or an API supported by none of the existing plugins. In that case you may need to create your own plugin.

## Custom plugins

EODAG has 5 categories of plugins `Search`, `Download`, `Authentication`, `Crunch` and `Api`.

Each provider must configure a `Search`, `Download` and an `Authentication` plugin.  
`Api` plugins can be used as an alternative to the 3 aforementioned plugins.

You can read more about plugins from the [EODAG plugins documentation page](https://eodag.readthedocs.io/en/latest/plugins.html).

We will now go through the creation of a sample custom Search plugin.

### 1. Create a custom Search plugin

A plugin can be installed as a separate python package.

1. Create a folder called `eodag_search_mock`. It will be the name of our package.

2. Make an empty ``__init__.py` file in the package folder.

3. Make a ```mock_search.py``` file and paste it the following MockSearch class definition. It is our search plugin.  
    The `query()`function is the one called by EODAG when searching for products.

    ```py
    # mock_search.py
    from typing import Any, List, Optional, Tuple
    from eodag.plugins.search import PreparedSearch
    from eodag.plugins.search.base import Search
    from eodag.api.product import EOProduct

    class MockSearch(Search):
      """Implement a Mock search plugin"""
      
      def query(
        self,
        prep: PreparedSearch = PreparedSearch(),
        **kwargs: Any,
      ) -> Tuple[List[EOProduct], Optional[int]]:
        """Generate EOProduct from the request"""
        return ([EOProduct(
            provider=self.provider,
            properties={
              **{
                  "id": f"mock_{kwargs.get('productType')}_{i}"
                },
              **kwargs
            }
          ) for i in range(0, prep.items_per_page)],
          prep.items_per_page if prep.count else None
        )


4. Create a `pyproject.toml` file in the package folder and paste in it the following content.
    The `projects.entry-points` is crucial for EODAG to detect this new plugin.

    ```toml
    [project]
    name = "eodag-search-mock"
    version = "0.0.1"
    description = "Mock Search plugin for EODAG"
    requires-python = ">=3.8"
    dependencies = [ "eodag" ]

    [project.entry-points."eodag.plugins.search"]
    MockSearch = "mock_search:MockSearch"
    ```

5. Your plugin is now ready. You need to install it for EODAG to be able to use it.
    ```shell
    pip install eodag_search_mock
    ```

### 2. Configure a provider with the new search plugin

In [5]:
import eodag

dag = eodag.EODataAccessGateway()

dag.update_providers_config("""
  demo_provider:
    search:
      type: MockSearch
      metadata_mapping:
    products:
      S2_MSI_L2A:
        productType: S2_MSI_L2A
      GENERIC_PRODUCT_TYPE:
        productType: '{productType}'
    download:
      type: HTTPDownload
    auth:
      type: GenericAuth
      credentials:
        password: xxxx
""")

sr = dag.search(productType="S2_MSI_L2A", provider="demo_provider")

sr

SearchResult([EOProduct(id=mock_S2_MSI_L2A_0, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_1, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_2, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_3, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_4, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_5, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_6, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_7, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_8, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_9, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_10, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_11, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_12, provider=demo_provider),
              EOProduct(id=mock_S2_MSI_L2A_13, provider=demo_provider),
  